### Conversation Buffer Memory

In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "Hello, world!"}, {"output": "Hi"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hello, world!'), AIMessage(content='Hi')]}

### Conversation Buffer Window Memory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=2 # 몇 개의 메시지를 저장할지
    )

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message("Hello", "Hi")
add_message("By", "By")
add_message("zzone", "ddeok")

In [12]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='By'),
  AIMessage(content='By'),
  HumanMessage(content='zzone'),
  AIMessage(content='ddeok')]}

### Conversation Summary Memory

In [16]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.4)
memory = ConversationSummaryMemory(llm=llm, return_messages=True)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("안녕, 나는 김쫀떡이야", "안녕!")
add_message("내 이름은 쫀떡궁합이라는 아이스크림의 이름에서 가져왔대", "너무 멋진 이름이야!")

get_history()

{'history': [SystemMessage(content='The human greets the AI in Korean, introducing themselves as Kim Jjondeok, which they say was inspired by an ice cream flavor. The AI responds with enthusiasm, calling it a cool name!')]}

### Conversation Summary Buffer Memory
> - Conversation Summary Memory + Conversation Buffer memory

In [22]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.4)
memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=150,
    return_messages=True
    )

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("안녕, 나는 김쫀떡이야", "안녕!")
add_message("내 이름은 쫀떡궁합이라는 아이스크림의 이름에서 가져왔대", "너무 멋진 이름이야!")

add_message("나는 디디라는 고양이 친구도 있어! 그 친구는 굉장히 덩치가 커!", "우와 너무 귀엽겠다!")
add_message("그리고 나는 루루라는 고양이 친구도 있는데 그 고양이는 나 못지 않게 식탐이 엄청나!", "루루도 엄청 귀여울 것 같아!")
get_history()

{'history': [SystemMessage(content='The human greets the AI in Korean, saying "Hello, I am Kim Jjondeok." The AI responds with "Hello!" and the human explains that their name is inspired by an ice cream called Jjondeok Gung-hap. The AI compliments the name as being really cool.'),
  HumanMessage(content='나는 디디라는 고양이 친구도 있어! 그 친구는 굉장히 덩치가 커!'),
  AIMessage(content='우와 너무 귀엽겠다!'),
  HumanMessage(content='그리고 나는 루루라는 고양이 친구도 있는데 그 고양이는 나 못지 않게 식탐이 엄청나!'),
  AIMessage(content='루루도 엄청 귀여울 것 같아!')]}

### Conversation Knowledge Graph Memory
> - 대화 중의 엔티티의 Knowledge graph를 만든다.
> - 가장 중요한 것들만 뽑아내는 요약본

In [38]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.4)
memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is Nicolas"})

add_message("Nicolas likes kimchi", "Wow that is so cool!")
memory.load_memory_variables({"inputs": "what does nicolas like"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea. Nicolas likes kimchi.')]}

### Memory on LLMChain
> - LLM chain = off-the-shelf-chain
> - 일반적인 목적을 가진 chain을 의미

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.4)
memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=100,
    memory_key="history_a",
    return_messages=True
    )

template = """
    당신을 사람을 도와주는 AI입니다.
    {history_a}
    Human: {quesition}
    You: 
"""

chain = LLMChain(
    llm=llm, 
    memory=memory, 
    prompt=PromptTemplate.from_template(template),
    verbose=True) # verbose=True로 설정하면, 각 단계에서 어떤 토큰이 생성되었는지 확인할 수 있다.

chain.predict(quesition="내 이름은 김쫀떡이야")
chain.predict(quesition="나는 대한민국에서 살고 있어")
chain.predict(quesition="내 이름이 뭐야?")


In [5]:
memory.load_memory_variables({})

{'history': [SystemMessage(content="The human introduces themselves as 김쫀떡. The AI responds warmly and finds the name cute and amusing, asking how it can help. The human mentions living in South Korea, and the AI expresses admiration for the country's rich history, culture, and delicious food, wishing the human a pleasant time."),
  HumanMessage(content='내 이름이 뭐야?'),
  AIMessage(content='죄송합니다, 저는 당신의 이름을 알 수 없습니다. 저에게 이름을 지어주시겠어요?')]}

### Chat Based Memory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.4)
memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=100,
    memory_key="history_a",
    return_messages=True
    )

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 사람을 도와주는 AI입니다."),
    MessagesPlaceholder(variable_name="history_a"),
    ("human", "{quesition}")
])

chain = LLMChain(
    llm=llm, 
    memory=memory, 
    prompt=prompt,
    verbose=True) # verbose=True로 설정하면, 각 단계에서 어떤 토큰이 생성되었는지 확인할 수 있다.

chain.predict(quesition="내 이름은 김쫀떡이야")
chain.predict(quesition="나는 대한민국에서 살고 있어")


In [12]:
chain.predict(quesition="내 이름이 뭐야?")



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 사람을 도와주는 AI입니다.
System: The human introduces themselves as Kim Jjeontteok. The AI responds with a greeting and asks how it can help Kim Jjeontteok.
Human: 나는 대한민국에서 살고 있어
AI: 대한민국에서 사는 김쫀떡님! 혹시 궁금한 것이 있나요? 도와드릴 내용이 있으면 언제든지 말씀해주세요.
Human: 내 이름이 뭐야?

> Finished chain.


'당신의 이름은 김쫀떡님입니다.'

### LCEL Based Memory

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.4)
memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=100,
    memory_key="history_a",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 사람을 도와주는 AI입니다."),
    MessagesPlaceholder(variable_name="history_a"),
    ("human", "{quesition}")
])


def load_memory(input):
    # print(input) # quesition이 들어옴 -> {'quesition': '내 이름은 김쫀떡이야'}
    # input 대신 _ (무시) 가능
    return memory.load_memory_variables({})["history_a"]
chain = RunnablePassthrough.assign(history_a=load_memory) | prompt | llm



def invoke_chain(question):
    result = chain.invoke({
        "quesition": question
    })
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)


invoke_chain("내 이름은 김쫀떡이야")



# chain.invoke({
#     # "history_a": memory.load_memory_variables({})["history_a"], # load_memory() 함수를 통해 history_a를 가져옴
#     "quesition": "내 이름은 김쫀떡이야"
# })

안녕하세요, 김쫀떡님! 무엇을 도와드릴까요?


In [20]:
invoke_chain("내 이름이 뭐야")

당신의 이름은 김쫀떡이라고 알고 있어요. 어떤 도움이 필요하신가요?
